# Carvana U-Net

## Imports

In [1]:
#from keras.optimizers import Adam
from keras.layers.advanced_activations import PReLU
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import AveragePooling2D
import tensorflow as tf
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from optimizers.AdamAccumulate import AdamAccumulate
from models.u_net import UNet
from submit import generate_submit
import utils

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## Preparing Data

In [2]:
input_size = 128
train_path = "inputs/train/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"
df_train = pd.read_csv('inputs/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])#[:3000]
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

print('Training on {} samples'.format(len(ids_train_split)))
print('Validating on {} samples'.format(len(ids_valid_split)))

bboxes = None
bbox_file_path = 'inputs/train_bbox.csv'
bboxes = utils.get_bboxes(bbox_file_path)

def train_generator(batch_size):
    return utils.train_generator(train_path, train_mask_path, ids_train_split, input_size, batch_size, bboxes)

def valid_generator(batch_size):
    return utils.valid_generator(train_path, train_mask_path, ids_valid_split, input_size, batch_size, bboxes)

Training on 4070 samples
Validating on 1018 samples


## Loss Functions

In [3]:
def dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#def bce_dice_loss(y_true, y_pred):
#    return binary_crossentropy(y_true, y_pred) + (1 - dice_value(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weights):
    return K.mean(tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, weights), axis=-1)

def weighted_dice_value(y_true, y_pred, weights):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    weights = K.flatten(weights)
    w2 = weights * weights
    return (2. * K.sum(w2 * intersection) + smooth) / (K.sum(w2 * y_true_f) + K.sum(w2 * y_pred_f) + smooth)

def weighted_bce_dice_loss(y_true, y_pred):
    a = AveragePooling2D(pool_size=(11, 11), strides=1, padding='same')(y_true)
    ind = K.cast(K.greater_equal(a, 0.01), 'float32') * K.cast(K.less_equal(a, 0.99), 'float32')
    ind = K.cast(ind, 'float32')
    weights = K.ones_like(a)
    w0 = K.sum(weights)
    weights = weights + ind*2
    w1 = K.sum(weights)
    weights = weights/w1*w0
    return  weighted_bce_loss(y_true, y_pred, weights) + (1 - weighted_dice_value(y_true, y_pred, weights))

## Create Model

In [4]:
#FC-DenseNet56:
#model = Tiramisu((input_size, input_size, 3), growth_rate=12, depth=5, layers_per_block=[4,4,4,4,4,4]) 
#FC-DenseNet67:
#model = Tiramisu((input_size, input_size, 3), growth_rate=16, depth=5, layers_per_block=[5,5,5,5,5,5]) 
#FC-DenseNet103:
#model = Tiramisu((input_size, input_size, 3), growth_rate=16, depth=5, layers_per_block=[4,5,7,10,12,15]) 
#FC-DenseNet46 (Not in paper):
#model = Tiramisu((input_size, input_size, 3), growth_rate=12, depth=4, layers_per_block=[4,4,4,4,5]) 

#U-Net:
model = UNet((input_size, input_size, 3), 
             activation=lambda x: PReLU()(x),
             dropout_base_only=True,
             init='he_uniform')
model.compile(optimizer=AdamAccumulate(accum_iters=4), loss=weighted_bce_dice_loss, metrics=[dice_value])

## Fit Model

In [ ]:
epochs = 50
batch_size = 16
run_name = utils.get_run_name('weights/', 'unet')
weight_path = 'weights/{}.hdf5'.format(run_name)

callbacks = [EarlyStopping(monitor='val_dice_value',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_value',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_value',
                             filepath=weight_path,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
             TensorBoard(log_dir='logs/{}'.format(run_name), batch_size=batch_size)]

#model.load_weights('weights/best_weights.hdf5')
#K.set_value(model.optimizer.lr, 0.01)

model.fit_generator(generator=train_generator(batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Epoch 1/50
255/255 [==============================] - 158s - loss: 0.5137 - dice_value: 0.9696 - val_loss: 0.5260 - val_dice_value: 0.9643
Epoch 2/50
255/255 [==============================] - 152s - loss: 0.4704 - dice_value: 0.9872 - val_loss: 0.4584 - val_dice_value: 0.9898
Epoch 3/50
255/255 [==============================] - 152s - loss: 0.4623 - dice_value: 0.9896 - val_loss: 0.4526 - val_dice_value: 0.9913
Epoch 4/50
255/255 [==============================] - 153s - loss: 0.4598 - dice_value: 0.9901 - val_loss: 0.4471 - val_dice_value: 0.9928
Epoch 5/50
255/255 [==============================] - 152s - loss: 0.4576 - dice_value: 0.9908 - val_loss: 0.5131 - val_dice_value: 0.9399
Epoch 6/50
255/255 [==============================] - 153s - loss: 0.4561 - dice_value: 0.9915 - val_loss: 0.4444 - val_dice_value: 0.9934
Epoch 7/50
255/255 [==============================] - 153s - loss: 0.4536 - dice_value: 0.9921 - val_loss: 0.4423 - val_dice_value: 0.9940
Epoch 8/50
255/255 [=======

## Validation

In [ ]:
def np_dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

### Prediction

In [ ]:
run_name = 'unet-2017-08-20-5'
model.load_weights('weights/{}.hdf5'.format(run_name))

val_imgs, val_masks = next(valid_generator(len(ids_valid_split)))
val_imgs = np.array(val_imgs)
val_masks = np.array(val_masks)
val_pred_masks = model.predict(val_imgs, batch_size=32)
masks_val_dices = [np_dice_value(mask, pred_mask) for (mask, pred_mask) in zip(val_masks, val_pred_masks)]

### Histogram

In [ ]:
hist, bins = np.histogram(masks_val_dices, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

### Visualization

In [ ]:
%matplotlib inline
index = 10
img_path = train_path.format(ids_valid_split.values[index])
utils.show_mask(img_path, 
                val_masks[index].squeeze(axis=-1),
                val_pred_masks[index].squeeze(axis=-1),
                show_img=True)

## Test

### Load Model

In [ ]:
# Create model first if required
run_name = 'unet-2017-08-20-5'
model.load_weights('weights/{}.hdf5'.format(run_name))

### Generate Submit

In [ ]:
orig_size = (1918, 1280)
batch_size = 2
threshold = 0.5
generate_submit(input_size, orig_size, batch_size, threshold, model, 'inputs/test/', 'submits/', run_name)